In [1]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import random
import matplotlib
from matplotlib import animation
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
import pandas as pd
from itertools import count
%matplotlib qt

In [9]:
scopes = ["https://www.googleapis.com/auth/yt-analytics.readonly"]
client_secrets_file = "./oauth.json"
# Disable OAuthlib's HTTPS verification when running locally.
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

In [10]:
# Get credentials and create an API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)

In [11]:
credentials = flow.run_console()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=499515213098-bg5fu39nhbsqph926do5ma59u7nf9dgt.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=5njIgVCZ7f6gt2bIrIBOJjfZ2bVWlG&prompt=consent&access_type=offline
Enter the authorization code: 4/1AfgeXvthEGJrfIM8tP4fStL__8va-FKwgCV-8Y2TxGqsg9I1gqCJnKVWo9Q


In [12]:
youtube_analytics = googleapiclient.discovery.build('youtubeAnalytics', 'v2', credentials=credentials)

In [13]:
def get_data():
    request = youtube_analytics.reports().query(
                    startDate='2021-04-24', 
                    endDate='2022-11-11',
                    ids='channel==UCOwyfb8QhvvXQpoQLJpmEKQ',
                    dimensions= 'day', 
                    metrics= 'views',
                    )
    response = request.execute()
    return response

In [14]:
data = get_data()

In [15]:
data.keys()

dict_keys(['kind', 'columnHeaders', 'rows'])

In [16]:
data['kind']

'youtubeAnalytics#resultTable'

In [17]:
data['columnHeaders']

[{'name': 'day', 'columnType': 'DIMENSION', 'dataType': 'STRING'},
 {'name': 'views', 'columnType': 'METRIC', 'dataType': 'INTEGER'}]

In [18]:
date = []
views = []
for d in data['rows']:
    date.append(d[0].replace('2022-',''))
    views.append(d[1])

In [128]:
last_30_days = date[-30:]
last_30_views = views[-30:]

In [129]:
len(last_30_days)

30

In [45]:
df = pd.DataFrame({"Date":last_30_days, "Views":last_30_views})

In [46]:
sns.set()
plot = sns.lineplot(data=df, x="Date", y="Views", label="Hacker-Davinci's Youtube Channel")
plt.setp(plot.get_xticklabels(), rotation=60)
plt.show()

## Animation

In [126]:
x,y= [], []
def animate_youtube(i):
    x.append(last_30_days[i])
    y.append((last_30_views[i]))
    plt.xticks(rotation = 60)
    plt.plot(x,y, scaley=True, scalex=True, color="blue")

In [127]:
fig = plt.figure(figsize=(10,5))
axes = fig.add_subplot(1,1,1)
# axes.set_ylim(0, 50)
axes.set_xlim(0, 30)
ani = FuncAnimation(fig=fig, func=animate_youtube, interval=200)
ani.save("test.gif")
plt.style.use("seaborn")

MovieWriter ffmpeg unavailable; using Pillow instead.


IndexError: list index out of range

MovieWriter ffmpeg unavailable; using Pillow instead.


IndexError: list index out of range

In [81]:
writergif = animation.PillowWriter(fps=1) 
ani.save(r"./views_animation.gif", writer=writergif)

IndexError: list index out of range

## 可以train 過去的資料，並預測明天多少人觀看